# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `findLargest(Mi: List[int], Ma: List[int]) -> float`

    A triangle is "inscribed" in a circle if all 3 points of the triangle are on the edge of the circle.  For this problem, our circle will be centered at the origin and have a radius of 5.  Our goal is to find the largest triangle (by area) we can inscribe in this circle.  Normally, this would be any equilateral triangle, but in this case we have the added restriction that each point of our triangle must be within one (or more) of the valid ranges of degrees.  The degree ranges are given in thousandths of degrees in corresponding elements of angleFrom and angleTo.  For each range, angleFrom will be less than or equal to angleTo and each will be between 0 and 360000.  All ranges are inclusive; see the examples for clarification.  Return the area of the largest inscribed triangle that can be made while following these restrictions.  If no triangle can be made, return 0.
    Notes
    -The ranges may overlap.
    -Your return value must have an absolute or relative error less than 1e-9.
 
    Constraints
    -angleFrom and angleTo will each contain between 1 and 30 elements, inclusive.
    -angleFrom and angleTo will contain the same number of elements.
    -Each element of angleFrom and angleTo will be between 0 and 360000, inclusive.
    -Each element of angleFrom will be less than or equal to the corresponding element of angleTo.
 
    Examples
    0)
        {0}
    {360000}

    Returns: 32.47595264191645
    We can use any point we want on the circle - so we can draw an equilateral triangle (which will be the biggest we can ever draw).

    1)
        {15000,25000,100000,265000,330000}
    {15000,25000,100000,265000,330000}

    Returns: 27.433829549752186
    In this case, each of our ranges are single points.  The biggest triangle can be made by using the points at 15°, 100°, and 265°.

    2)
        {245900,246500,249900}
    {245915,246611,252901}

    Returns: 0.002789909594714814
    We only have 3 small ranges, all near to each other - so our best triangle is still really small.

    3)
        {42}
    {42}

    Returns: 0.0
    It's hard to draw a triangle with one point.

    

The only idea that I have is a brute force algorithm that tries all possible angles.
However, the brute force algorithm is really bad because there are up to $360000\times(360000-1)\times(360000-2)$ possibilities.

Since there are so many possible states and the cost of calculating the area is very small, the only alternative I know is to find a clever mathematical way of finding the solution.
Unfortunately, I do not have any idea or justification of an algorithm that can do so,
therefore I will implement a brute force algorithm and try to optimize it.

Lets start with a function that calculates all possible angles.

In [1]:
def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

assert len(possible_angles([0], [360000])) == 360001
assert len(possible_angles([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])) == 5
print('good')

good


The function seems to be working.

Now I will implement a function that calculates the area.
The calculation of the area will convert the tree angles into (x,y) coordinates, and use the cross product to calculate the area.

After, I will implement a brute force algorithm that, given an initial angle, will return the maximum area of the triangle that has a vertice in that angle.
The brute force algorithm will walk vertices $b$ and $c$ away from the starting point $a$.
At each iteration, I will calculate the area of the triangle $abc$, by walking $b$, $c$ and $b$ and $c$ away from $a$.
If the area increases, I will update the values of $b$ and $c$ and keep the iteration.
If the area decreases, I will return the maximum area.
This algorithm supposes that there is only one maximum.

In [2]:
from typing import List
from math import sin, cos

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(ta/1000), 5*sin(ta/1000)
    xb, yb = 5*cos(tb/1000), 5*sin(tb/1000)
    xc, yc = 5*cos(tc/1000), 5*sin(tc/1000)
    
    area = xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb
    
    return 0.5*area

def optimize_area(a, possible_angles):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(possible_angles)
    
    a_idx = possible_angles.index(a)
    b_idx = (a_idx-1+pa_size)%pa_size
    c_idx = (a_idx+1        )%pa_size
    
    area = calculate_area(a, possible_angles[b_idx], possible_angles[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, possible_angles[next_b], possible_angles[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, possible_angles[b_idx], possible_angles[next_c])
        
        area_bc = calculate_area(a, possible_angles[next_b], possible_angles[next_c])
        
        if area_bc > next_b and area_bc > next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            b_idx = next_c
            area = area_c
            
        else:
            break
            
    return area
            
pa = possible_angles([0], [360000])
optimize_area(0, pa)

1.0107520213159527e-08

The area is incorrect, it shoud return 32.47595264191645.

I will add a a print statement to print the areas and the angles.

In [3]:
from typing import List
from math import sin, cos

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(ta/1000), 5*sin(ta/1000)
    xb, yb = 5*cos(tb/1000), 5*sin(tb/1000)
    xc, yc = 5*cos(tc/1000), 5*sin(tc/1000)
    
    area = xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb
    
    return 0.5*area

def optimize_area(a, possible_angles):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(possible_angles)
    
    a_idx = possible_angles.index(a)
    b_idx = (a_idx-1+pa_size)%pa_size
    c_idx = (a_idx+1        )%pa_size
    
    area = calculate_area(a, possible_angles[b_idx], possible_angles[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, possible_angles[next_b], possible_angles[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, possible_angles[b_idx], possible_angles[next_c])
        
        area_bc = calculate_area(a, possible_angles[next_b], possible_angles[next_c])
        print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc > next_b and area_bc > next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            b_idx = next_c
            area = area_c
            
        else:
            break
            
    return area
            
pa = possible_angles([0], [360000])
optimize_area(0, pa)

-0.016040142746453157 360000 1 -0.01602815276076619 -0.03206828300722009 -0.032044299507688834
-0.01602815276076619 359999 1 -0.016016159246923678 -0.032044299507688834 -0.032020308963842226
-0.016016159246923678 359998 1 -0.016004162216919582 -0.032020308963842226 -0.03199631139967174
-0.016004162216919582 359997 1 -0.015992161682751416 -0.03199631139967174 -0.03197230683916885
-0.015992161682751416 359996 1 -0.015980157656420246 -0.03197230683916885 -0.03194829530634813
-0.015980157656420246 359995 1 -0.015968150149928917 -0.03194829530634813 -0.03192427682521526
-0.015968150149928917 359994 1 -0.015956139175287376 -0.03192427682521526 -0.03190025141978836
-0.015956139175287376 359993 1 -0.01594412474450202 -0.03190025141978836 -0.03187621911409444
-0.01594412474450202 359992 1 -0.015932106869591678 -0.03187621911409444 -0.03185217993216405
-0.015932106869591678 359991 1 -0.01592008556257163 -0.03185217993216405 -0.0318281338980384
-0.01592008556257163 359990 1 -0.01590806083546603 -

1.0107520213159527e-08

I realized that I forgot to get the absolute value of the cross product.

In [4]:
from typing import List
from math import sin, cos, pi

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, possible_angles):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(possible_angles)
    
    a_idx = possible_angles.index(a)
    b_idx = (a_idx-1+pa_size)%pa_size
    c_idx = (a_idx+1        )%pa_size
    
    area = calculate_area(a, possible_angles[b_idx], possible_angles[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, possible_angles[next_b], possible_angles[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, possible_angles[b_idx], possible_angles[next_c])
        
        area_bc = calculate_area(a, possible_angles[next_b], possible_angles[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        else:
            break
            
    return area
            
pa = possible_angles([0], [360000])
optimize_area(0, pa)

32.47595264191645

The result is correct now.

The case above is one of the worst cases for this algorithm. Lets mesure the time.

In [6]:
from datetime import datetime

t1 = datetime.now()
assert optimize_area(0, pa) == 32.47595264191645
t2 = datetime.now()
print(t2 - t1)

0:00:00.327408


The algorithm runs fine if we know for sure one of the angles that maximizes the area.
However, this is not the case for every set of Mi and Ma.

Lets try to optimize by walking $a$ and $b$ in larger steps.

In [7]:
from typing import List
from math import sin, cos, pi

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, possible_angles):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(possible_angles)
    
    a_idx = possible_angles.index(a)
    
    step = len(possible_angles)//4 if len(possible_angles)//4 > 0 else 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, possible_angles[b_idx], possible_angles[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, possible_angles[next_b], possible_angles[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, possible_angles[b_idx], possible_angles[next_c])
        
        area_bc = calculate_area(a, possible_angles[next_b], possible_angles[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area

from datetime import datetime
    
t1 = datetime.now()
assert optimize_area(0, pa) == 32.47595264191645
t2 = datetime.now()
print(t2 - t1)

0:00:00.327058


Seems a little bit better, but probabilly not enough.

Let me test a little bit more, and I will try to optimize it later.

In [8]:
t1 = datetime.now()
pa = possible_angles([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])
print(optimize_area(15000, pa))
print(27.433829549752186)
t2 = datetime.now()

print(t2 - t1)

27.433829549752183
27.433829549752186
0:00:00.006630


The result above is correct.

Let me finish the brute force algorithm.
The brute force algorithm will simply call optimize_area for every possible starting angle

In [9]:
def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    for a in angles:
        
        area = optimize_area(a, possible_angles)
        
        if area > max_area:
            max_area = area
            
    return area

def test():
    assert (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000]))==27.433829549752186,a

test()

TypeError: object of type 'function' has no len()

The function and the variable have the same name.

I will change the variable name

In [10]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = len(pa)//4 if len(pa)//4 > 0 else 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    for a in angles:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return area

def test():
    assert (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000]))==27.433829549752186,a

test()

AssertionError: 12.73791784323853

I should return max_area instead of area

In [11]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = len(pa)//4 if len(pa)//4 > 0 else 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    for a in angles:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return max_area

def test():
    assert (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000]))==27.433829549752186,a

test()

AssertionError: 27.433829549752183

The error must be withing 1e-9.
I will use numpy.isclose to compare the numbers.

In [12]:
from numpy import isclose

def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')

test()

its working


Let me add one more test 

In [13]:
def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
test()

AssertionError: 0.0025385848964418756

The result is close, but not quite there.
The problem may be that we are walking in large steps, and that we are jumping the optimal angle.
Lets set the step = 1.

In [14]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = len(pa)//4 if len(pa)//4 > 0 else 1
    step = 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-1+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+1        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    for a in angles:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return max_area

def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
t1 = datetime.now()
test()
t2 = datetime.now()
print(t2 - t1)

its working
0:00:50.230936


Now that I reviewed the code, my steps implementation is wrong.
I will fix it, however the corectness of the example above showed me that I must use step =1.
Therefore I will set step = 1 and try to optimize the problem later.

Let me add a test where it is not possible to for a triangle. 

In [15]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = len(pa)//4 if len(pa)//4 > 0 else 1
    step = 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-step+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+step        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
#         print(area, possible_angles[b_idx], possible_angles[c_idx], area_b, area_c, area_bc)
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    to_try = []
    for a, b in zip(Mi, Ma):
        to_try += [a, b, (a+b)//2]
        if (a+b)//2+1 <= b:
            to_try += [(a+b)//2+1]
        
    
    for a in to_try:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return max_area

def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([42], [42])),
        0., rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
t1 = datetime.now()
test()
t2 = datetime.now()
print(t2 - t1)

KeyboardInterrupt: 

Looks like it is running indefinitely.
Therefore I have the add the case where there are less than 3 distinct points as a special case.

Also, I came with an idea to optimize the problem.
The ideia is that the triangle must be somewhat symmetrical to have a maximum area.
Therefore, one of the edges must be in the middle of each Mi[i] and Ma[i].
If it is not possible to be there, it should be because the angle that makes the point symmetrical is not available, and the angle maximizes the area is the closest to it: either Mi[i] or Ma[i].

With this in mind, I will change the starting angles to be Mi[i],Ma[i], and their average.

In [16]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-step+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+step        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    max_area = 0
    
    to_try = []
    for a, b in zip(Mi, Ma):
        to_try += [a, b, (a+b)//2]
        if (a+b)//2+1 <= b:
            to_try += [(a+b)//2+1]
        
    
    for a in to_try:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return max_area

def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([42], [42])),
        0., rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
t1 = datetime.now()
test()
t2 = datetime.now()
print(t2 - t1)

KeyboardInterrupt: 

I forgot to fix the loop.

In [17]:
from typing import List
from math import sin, cos, pi

def possible_angles(Mi, Ma):
    pa = list()
    
    for min_a, max_a in zip(Mi, Ma):
        pa += [a for a in range(min_a, max_a+1)]
        
    return sorted(list(set(pa)))

def calculate_area(ta, tb, tc):
    xa, ya = 5*cos(2*pi*ta/360000), 5*sin(2*pi*ta/360000)
    xb, yb = 5*cos(2*pi*tb/360000), 5*sin(2*pi*tb/360000)
    xc, yc = 5*cos(2*pi*tc/360000), 5*sin(2*pi*tc/360000)
    
    area = abs(xa*yb + xb*yc + xc*ya - xa*yc - xb*ya - xc*yb)
    
    return 0.5*area

def optimize_area(a, pa):
    
    xa, ya = 5*cos(a/1000), 5*sin(a/1000)
    pa_size = len(pa)
    
    a_idx = pa.index(a)
    
    step = 1
    
    b_idx = (a_idx-step+pa_size)%pa_size
    c_idx = (a_idx+step        )%pa_size
    
    area = calculate_area(a, pa[b_idx], pa[c_idx])
    
    while True:
        
        next_b = (b_idx-step+pa_size)%pa_size
        area_b = calculate_area(a, pa[next_b], pa[c_idx])
        
        next_c = (c_idx+step        )%pa_size
        area_c = calculate_area(a, pa[b_idx], pa[next_c])
        
        area_bc = calculate_area(a, pa[next_b], pa[next_c])
        
        if area_bc >= next_b and area_bc >= next_c and area_bc > area:
            b_idx, c_idx = next_b, next_c
            area = area_bc
            
        elif area_b >= area and area_b >= area_c:
            b_idx = next_b
            area = area_b
            
        elif area_c >= area and area_c >= area_b:
            c_idx = next_c
            area = area_c
            
        elif step == 1:
            break
            
        else:
            step = step//2 if step//2 > 0 else 1
            
    return area


def findLargest(Mi: List[int], Ma: List[int]) -> float:
    
    angles = possible_angles(Mi, Ma)
    
    max_area = 0
    
    if len(angles) < 4:
        return 0
    
    start = []
    for a, b in zip(Mi, Ma):
        start += [a, b, (a+b)//2]
        if (a+b)//2+1 <= b:
            start += [(a+b)//2+1]
        
    for a in start:
        
        area = optimize_area(a, angles)
        
        if area > max_area:
            max_area = area
            
    return max_area

def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([42], [42])),
        0., rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
t1 = datetime.now()
test()
t2 = datetime.now()
print(t2 - t1)

its working
0:00:00.073148


It is working and it is very fast.
Let me add the last case.

In [18]:
def test():
    assert isclose(
        (a:=findLargest([15000,25000,100000,265000,330000], [15000,25000,100000,265000,330000])),
        27.433829549752186, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([245900,246500,249900], [245915,246611,252901])),
        0.002789909594714814, rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([42], [42])),
        0., rtol=1e-9, atol=1e-9
        ),a
    assert isclose(
        (a:=findLargest([0], [360000])),
        32.47595264191645, rtol=1e-9, atol=1e-9
        ),a
    
    print('its working')
    
t1 = datetime.now()
test()
t2 = datetime.now()
print(t2 - t1)

its working
0:00:05.431634


The code seems to be working.
There may be other cases that the function will fail, but I cannot think of any so far.